# I/O and Advanced Fundamentals

- Basic stream I/O
- Vector and Matrix operations
- Perceptron - Part 1
- Perceptron - Part 2

## 1. Basic Stream I/O

- Stream I/O
- Formatted output using @printf([fid,]"format",args)
- fid is the file handle (Stream I/O) which works for formatted strings using s=@sprintf("format",args)
- Some handy OS commands

In [1]:
# well depth example with file i/o
#  constants
using Printf
g= 9.807    # accel due to gravity
vs= 343.21  # velocity of sound in air in m/s 20 degrees Celsius
# (depth, tv)=well(t)  judging well depth using a rock and stopwatch
function well(t)
    tdn = (-vs + sqrt(vs^2 + 2.0*g*vs*t))/g
    tup = t - tdn
    depth = vs*tup; termvel=g*tdn
    return (depth, termvel)
end

well (generic function with 1 method)

In [2]:
t = 5.0 # seconds from drop to ear
(d,tv) = well(t)
println(t," ",d," ",tv)

5.0 107.68507177482142 45.95796990502703


In [3]:
# rounding results:
rd =(round(d*10))/10.0
rtv =(round(tv*10))/10.0

println("depth $rd meters")
println("terminal velocity $rtv meters/second")

depth 107.7 meters
terminal velocity 46.0 meters/second


In [5]:
# could also ue @printf() macro for C printf
@printf("depth %6.1f meters",d)

depth  107.7 meters

In [6]:
@printf("%6.1f,%6.1f,%6.1f\n",t,d,tv)

   5.0, 107.7,  46.0


In [8]:
win=open("./files/wellin.dat","r")
wout=open("./files/wellout.csv","w")

@printf(wout,"   Time Depth Speed Place\n")
@printf(wout,"   sec    M    M/s  CA/USA\n")
while true
    s=readline(win)
    if(s==""); break; end
    st=split(s)
    t=parse(Float64, st[1])
    (d,tv) = well(t)
    @printf(wout,"%6.1f,%6.1f,%6.1f, %-22s\n", t,d,tv,st[2])
end
println("done")
close(win)
close(wout)

done


In [9]:
# many system-like directory functions
# examples:
pwd()

"/Users/henrique/Desktop/julia"

In [10]:
readdir() # list of files/directories in working directory

9-element Array{String,1}:
 ".DS_Store"
 ".git"
 ".ipynb_checkpoints"
 "1-Getting Started.ipynb"
 "2-Understanding Julia Fundamentals.ipynb"
 "3 - Control Structures, Functions, Tasks and Modules.ipynb"
 "4 - IO and Advanced Fundamentals.ipynb"
 "README.md"
 "files"

In [11]:
stat("./files/wellin.dat") #yields string, list of file parameters
# mode=6 Owner R&W, 
#     =4 Group R only,
#     =4 Others R only
# uperm(file) user permission, gperm(file) group permission
# operm(file) other permission

StatStruct(mode=0o100644, size=115)

In [12]:
chmod("./files/wellout.csv", 0o664)
stat("./files/wellout.csv")

StatStruct(mode=0o100664, size=368)

#### Takeaways

- Opening and Closing streams
- Formatted output
- Format control characters
- a few OS commands

## 2. Vector and Matrix Operations

- Vector and Matrix Operations:
    - Normal Math ops and functions do what you would expect
- Dot notation:
    - .+ , .- , .* , .^ , .% , ./ , .\ perform operations on corresponding elements
- Two common matrix decompositions:
    - Sparse matrices
    
Most of the math functions work equally well on Vectors and Matrices.  
The same goes for operations, but some need a preceding dot (.) to be corresponding element by element operations.  
Those include: .+ , .- , .* , .^ , .% , ./ and .\ or any operation with otherwise different normal vector/matrix behavior.

In [14]:
using LinearAlgebra

A = [2.5 2 -4; 3 -1 6; -10 2.3 4.4]

println(eigvals(A))
println(eigvecs(A))

[-6.617090724982948, 2.7882910325743167, 9.728799692408636]
[-0.3932792609247248 0.26696075560426874 0.36792566733230614; 0.7610668200252224 0.8695248970921453 -0.3733716151421295; -0.5158572655136283 0.41552185057357915 -0.8516010452813187]


In [15]:
# three ways to get an inverse
Ainv = inv(A)
A^-1
A\I  # I is the identity/uniform-scaling operator

3×3 Array{Float64,2}:
 0.101393   0.100279  -0.0445682
 0.407799   0.16156    0.150418
 0.0172702  0.143454   0.0473538

In [16]:
A*Ainv

3×3 Array{Float64,2}:
  1.0          0.0          -2.77556e-17
 -2.77556e-17  1.0           0.0
  8.32667e-17  1.11022e-16   1.0

In [17]:
Am1 = A^2 .+ exp.(A) .- 1

3×3 Array{Float64,2}:
  63.4325   0.189056  -16.5817
 -36.4145  20.1679    410.829
 -63.1     -3.20582   153.611

In [18]:
B = [2.5 2 -4; 2 -1 -3; -4 -3 4.4]
sB = Symmetric(B) # less computation

3×3 Symmetric{Float64,Array{Float64,2}}:
  2.5   2.0  -4.0
  2.0  -1.0  -3.0
 -4.0  -3.0   4.4

In [21]:
# solve for x in sB+x=v linear system of equations
v = [1., 2, 3]

x = sB\v

println(x) #linear solution

[-2.868217054263566, -0.6434108527131782, -2.364341085271318]


In [22]:
# more than 12 different factorizations are available
# Let's use Single Valued Decomposition SVD
(U,S,V) = svd(B)  # such that U*diagm(S)*V'
println(U)  # Matrix
println(V)  # Matrix
println(S)  # Vector diagonal of Matrix

[-0.574342923296777 -0.08332196272978316 -0.8143633445740107; -0.34236464703591796 0.9280752576617368 0.14650175622180192; 0.7435836569740379 0.36295146596061256 -0.5615599508851739]
[-0.5743429232967773 0.08332196272978323 0.8143633445740107; -0.34236464703591807 -0.9280752576617368 -0.14650175622180206; 0.7435836569740379 -0.3629514659606127 0.5615599508851736]
[8.87086969047107, 2.3527979686740075, 0.6180717217970606]


In [24]:
println(U*Diagonal(S)*V')  # V' transpose of matrix V

[2.500000000000001 2.0 -3.9999999999999996; 2.0000000000000004 -0.9999999999999999 -3.000000000000001; -4.000000000000002 -3.0000000000000013 4.4]


In [25]:
# LU decomposition L= lower triangular part,
# U= upper triangular part, p= permutation vector
(L,U,p)=lu(B)
println(L) # lower triangle
println(U) # upper triangle
println(p) # permutation vector for rows
println((L*U)[p,:]) # rearrange rows by p

[1.0 0.0 0.0; -0.5 1.0 0.0; -0.625 -0.05 1.0]
[-4.0 -3.0 4.4; 0.0 -2.5 -0.7999999999999998; 0.0 0.0 -1.29]
[3, 2, 1]
[2.5 2.0 -4.0; 2.0 -1.0 -3.0; -4.0 -3.0 4.4]


In [26]:
# make a random matrix
R = rand(3,3) .+ Diagonal([1.0, 1, 1])
Ri = inv(R)
println(R)
println()
println(Ri)

[1.708827506211986 0.08102477381861384 0.931250274356308; 0.5907052265122514 1.2165433112917443 0.20003987645520271; 0.7599736265487387 0.9004496463598017 1.5501802678202217]

[0.6857821936154275 0.2866342389036346 -0.44896270978199904; -0.3070314133703831 0.7804755863970717 0.08373016400248426; -0.15785909432188297 -0.5938750782132266 0.8165534347514273]


In [28]:
# Sparse matrices
using SparseArrays

#            row index     col index     values   
S=sparse([1, 2, 3, 3], [1, 3, 1, 2], [3, 3, 2, 0])
S=dropzeros(S)

3×3 SparseMatrixCSC{Int64,Int64} with 3 stored entries:
  [1, 1]  =  3
  [3, 1]  =  2
  [2, 3]  =  3

In [29]:
R=S*I

3×3 SparseMatrixCSC{Int64,Int64} with 3 stored entries:
  [1, 1]  =  3
  [3, 1]  =  2
  [2, 3]  =  3

In [30]:
R*rand(3,3)

3×3 Array{Float64,2}:
 1.67069    2.59028  0.2596
 0.0507147  1.98468  1.62814
 1.1138     1.72686  0.173067

#### Takeaways

- Learned more about Vector and Matrix operations
- Learned when to use . notation
- Learned matrix decomposition - 2 examples
- Learned Sparse Matrices

## 3. Perceptron - Part 1

- Perceptron and its limitations
- How to prepare data for a learning network

### Perceptron

- The perceptron is a one layer neural network
- The one we are using can be described with the function:
    - y = f(sum(x[i] * w[i])+bias)
    - y = 1 if x * w + bias > 0
    - y = 0, otherwise
    
### Limitations of Single Layer Perceptrons

- A 1-layer perceptron cannot implement the XOR function
- Relevant data must be separable by a hyperplane
- Limitations of the Perceptron nearly killed neural networks as a field of study
- Do you have enough data and does it represent the whole?

### Data Transformation, Representation, and Randomization

- The problem at hand is to identify a mine from sonar data
- Each sonar record contains the correct M (mine) or R (rock) identifier at the end, however, we need to separate these
- The data is clumped, all mines together, all rocks together
- We need to randomizee these, select a training group and a testing group
- Each training epoch (run) needs to randomize the data

In [34]:
# using UCI Machine Learning laboratory data
# you need to know your training/test data
# it generally doesn't come in a ready-to-use form
# This sonar data is 60 magnitudes in a time sequence
#    followed by a letter M=mine and R=rock
# read in training data
# Create Sonar Data Set
using Random

numsonar=208  # total number of sonar sweeps
trainln=138   # training length after initial randomization
burstln=60    # sonar burst length
testln=numsonar-trainln  # 70 is test length

# Some arrays
traindata=Array{Float64}(undef,trainln,burstln)
trainlabel=Array{Int32}(undef,trainln)
testdata=Array{Float64}(undef,testln,burstln)
testlabel=Array{Int32}(undef,testln)
#

function create_sonar_data_arrays()
    global traindata, trainlabel, testdata, testlabel
    sdata=Array{Float64}(undef,numsonar,burstln)
    stype=Array{Int32}(undef,numsonar)
    sonardata=zeros(Float64, burstln)
    f1=open("./files/sonar-data.csv","r")
    for i=1:numsonar
        sonarlist=split(readline(f1),",")
        stype[i]=Char(sonarlist[burstln+1][1])
        sdata[i,:]=parse.(Float64, sonarlist[1:burstln])
    end
    # randomize and partition training from test data
    seq=randperm(numsonar) # random index sequence
    for i=1:numsonar
        sonardata[:]=sdata[seq[i],:]
        sonartype=(stype[seq[i]]==Int32('M')) ? 1 : 0
        if i <=trainln
            traindata[i,:]=sonardata
            trainlabel[i]=sonartype
        else
            testdata[i-trainln,:]=sonardata
            testlabel[i-trainln]=sonartype
        end
    end
    close(f1)
end

create_sonar_data_arrays()

In [35]:
println(trainlabel)

Int32[1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1]


#### Takeaways

- A bit about AI and the perceptron (an early stage of Neural Network)
- How to organize training and test data (in general) for neural networks
- Check that your data transformations worked, however the best test is to transform it back to the original and compare it

## Perceptron - Part 2

- How to train a perceptron
- The training formula
- Constructing training epochs
- Training and testing in Julia code

### Training formula

- It amounts to feeding back the error in proportion to inputs on a wrong result
- A pass through the training samples is called an epoch
- Weights and bias are initialized to 0.0
- Order of training samples (set of inputs with labels) randomized (for each epoch)
- For each sample fed through network p=(inputs' * w + bias > 0) ? 1 : 0  # p is prediction
- if p doesn't match the label then  
    - error=label-p and serror+=error^2 (Minimize squared error sum: serror)  
    - w.+= learning_rate * error .* inputs  
    - bias += learning_rate * error  
  else do nothing

In [44]:
# Training module
inputsets = traindata
labels = trainlabel

# Parameters
epochs=2000
global learning_rate=0.00045
global bias = 0.0
global weights=zeros(Float64, burstln)

# Prediction function
predict(sinput, weights, bias) = sinput' * weights + bias > 0.0 ? 1 : 0

# training
function train(inputsets, weights, bias)
    sinput=zeros(Float64, burstln) # will be sonar input sample
    save_weights=zeros(Float64, burstln)
    save_bias=0.0
    last_serror=70
    for i=1:epochs                 # loop through epochs
        score=0
        error=0.0
        serror=0.0
        seq=randperm(trainln)
        lr=learning_rate
        for j=i:trainln
            sinput[:] = inputsets[seq[j],:]
            p = predict(sinput, weights, bias)
            if p != labels[seq[j]]
                error=labels[seq[j]]-p
                serror+=error^2
                weights .+= lr * error .* sinput
                bias += lr * error
            else
                score+=1
            end
        end
        if serror<last_serror
            @printf("epoch=%4d, perscore=%5.1f, serror=%5f1\n",i,score*100.0/trainln,serror)
            last_serror=serror
            save_weights=weights
            save_bias=bias
        end
    end
    return (save_weights, save_bias)
end

(weights, bias) = train(inputsets, weights, bias)

epoch=   1, perscore= 58.0, serror=58.0000001
epoch=   2, perscore= 67.4, serror=44.0000001
epoch=   4, perscore= 68.8, serror=40.0000001
epoch=   6, perscore= 76.8, serror=27.0000001
epoch=   9, perscore= 76.1, serror=25.0000001
epoch=  13, perscore= 73.9, serror=24.0000001
epoch=  21, perscore= 68.8, serror=23.0000001
epoch=  22, perscore= 68.8, serror=22.0000001
epoch=  24, perscore= 68.8, serror=20.0000001
epoch=  32, perscore= 65.9, serror=16.0000001
epoch=  47, perscore= 57.2, serror=13.0000001
epoch=  61, perscore= 47.8, serror=12.0000001
epoch=  66, perscore= 47.1, serror=8.0000001
epoch=  69, perscore= 46.4, serror=6.0000001
epoch=  86, perscore= 34.8, serror=5.0000001
epoch= 100, perscore= 25.4, serror=4.0000001
epoch= 102, perscore= 24.6, serror=3.0000001
epoch= 108, perscore= 21.0, serror=2.0000001
epoch= 122, perscore= 12.3, serror=0.0000001


([0.001973159999999993, 0.001172024999999997, 0.0024649200000000033, 0.005333624999999998, 0.0032376600000000094, 7.577999999999785e-5, -0.004627935000000019, -0.006079860000000019, 0.005152454999999998, 0.0022351499999999974  …  0.0014480100000000043, 0.0017460899999999922, 0.0009405900000000054, 0.0006279750000000009, -0.00027998999999999976, 0.0007069050000000005, -0.000394695, 0.0010205099999999983, 0.0013204349999999937, 0.0010950300000000014], -0.009000000000000001)

In [45]:
function test(inputsets, weights, bias)
    sinput=zeros(Float64, burstln)
    error=0
    serror=0
    score=0
    for j=1:testln
        sinput[:] = inputsets[j,:]
        p = predict(sinput, weights, bias)
        if p != labels[j]
            error=labels[j]-p
            serror+=error^2
        else
            score+=1
        end
    end
    @printf("perscore=%5.2f, serror=%3d\n",score*100.0/testln,serror)
end

test(inputsets, weights, bias)

perscore=84.29, serror= 11


#### Takeaways

- How to build a simple neural net and train it and some thoughts along the way
- Julia is a very expressive language, and some of the examples have been condensed by using language elements and features not introduced earlier, the implicit: for loop for example
- Keep on reading and experimenting with the language